# Authentication - Rest API - OAuth DeviceCode

## Import credential from storage path or Export a new XML file using above parameters

In [2]:
$config = @{
    tenantDomain = "domain.com"
    tenantID = "xxxxxxx-xxxx-xxxx-xxxxx-xxxxxxxxxx"          ## Azure AD Tenant ID
    appID = "xxxxxx-23af-4ae5-a6ed-xxxxxx"                   ## Usually known as Client ID or Application ID
    appSecret = "W_xxxxxxxxxxxxxxxxxx.xxxxxxxxx"             ## This is the secretKey. Important only authorized users have this 
    resourceURI = "https://graph.microsoft.com"   ## This is the resource endpoint for MDATP
}

In [ ]:
## Checks to see if config file exist and if not export credential file from above.
$storagePath = ".\graph.credential"
$config = (Import-CliXml -Path $storagePath)
if (!$config) {
    $config | Export-CliXml -Path $storagePath
} else {
    Write-Host -Foregroundcolor green "`nCredential file loaded from $($storagePath)"
}

## Verify if parameters look good

In [ ]:
$authUrl = "https://login.microsoftonline.com/$($config.tenantDomain)";
Write-Host -ForeGroundColor Yellow "Authentication Parameters"
Write-Host "Resource URL: $($config.resourceURI)"
Write-Host "Client APP ID: $($config.AppId)"
Write-Host "AuthUrl: $($authUrl)"

$postParams = @{ 
    resource = $config.resourceURI 
    client_id = $config.AppID
}
$postParams

## Make Rest API Call and authenticate via Device Code

In [ ]:
$postParams = @{ 
    resource = $config.resourceURI 
    client_id = $config.AppID
}
$tokenResponse = $null; 
$response = Invoke-RestMethod -Method POST -Uri "$authurl/oauth2/devicecode" -Body $postParams
$tokenParams = @{ grant_type = "device_code"; resource = $config.resourceURI; client_id = $config.AppID; code = "$($response.device_code)" }
Write-Host $response.message
#I got tired of manually copying the code, so I did string manipulation and stored the code in a variable and added to the clipboard automatically
$code = $response.message -match "code\s(.+)\sto"
$code = $Matches[1]
Set-Clipboard -Value $code

Write-Host "Waiting for code"
While (!$tokenResponse) {
    Try {
        $tokenResponse = Invoke-RestMethod -Method POST -Uri "$authurl/oauth2/token" -Body $tokenParams -ErrorAction Ignore
        Write-Host -ForeGroundColor Green "`nReceived Token!"
        Write-Host -ForegroundColor Green "Connected and Access Token received and will expire $($tokenResponse.expires_on)"
    } Catch {
    }
}

## Check if code valid

In [ ]:
$tokenResponse | Select-Object -Property @('token_type','scope','expires_on','resource') | FL

In [ ]:
$howManyDays = 7   # How many days do you want to go back
$reportFormat = 'application/json'   #report Format 

## Request Parameters
$URI = "https://graph.microsoft.com/beta/reports/getEmailActivityCounts(period='D30')?`$format=application/json"  ## URL of where to query in the graph API/beta/reports/getEmailActivityCounts(period='D7')?$format=text/csv
$URI
$authHeader = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" 
}


In [ ]:
(Invoke-RestMethod -Method Get -Uri $URI -Headers $authHeader -ErrorAction Stop)

In [ ]:
#query Graph API and insert into $graphresponse variable
$Result = (Invoke-RestMethod -Method Get -Uri $URI -Headers $authHeader -ErrorAction Stop).value
$Result | Select -First 5 | Format-Table

## Clean up any empty counts in send and receive and replace with '0'

In [ ]:
ForEach ($item in $Result) {
    if ($item.send -eq $null) {
        $item.send = 0
    }
    if ($item.receive -eq $null) {
        $item.receive = 0
    }
}

### Generate Scatter Report based off email Activity

In [ ]:
$sendSeries = [Graph.Scatter]@{
    name = "Emails Sent"
    x = $Result.reportDate
    y = $Result.send
}

$receiveSeries = [Graph.Scatter]@{
    name = "Emails Received"
    x = $Result.reportDate
    y = $Result.receive
}

$chart = @($receiveSeries, $sendSeries) | New-PlotlyChart -Title "Emails Received vs Sent"
Out-Display $chart